Imports

In [1]:
using LogicCircuits
using ProbabilisticCircuits
using DataFrames
using BenchmarkTools
using TikzPictures
using Statistics: mean
using LinearAlgebra: normalize
using Clustering: kmeans, nclusters, assignments
TikzPictures.standaloneWorkaround(true)
import Printf
using LogicCircuits.LoadSave: SDDElement, 
    PSDDElement, 
    save_lines,
    get_vtree2id,
    get_node2id,
    parse_psdd_file, 
    PsddHeaderLine, 
    LcHeaderLine, 
    get_nodes_level

In [2]:
NCLUSTERS = 3
MINCLUSTERSIZE = 3
LOGS = true
VERBOSE = true

true

Utilities

In [3]:
function bs2arr(bs)
    y = variables(bs) # parsing the indexes of the variables on the right DEBUG: case of a single var
    yy = (x->collect.(x)).(y)
    b = zeros(Int8,size(yy,1))
    for i = 1:size(yy,1)
        b[i] = yy[i][1]
    end
    return b
end

function lr(vtree)
    if typeof(vtree.left) == PlainVtreeLeafNode && typeof(vtree.right) == PlainVtreeLeafNode
        l = [Int16(variable(vtree.left))]
        r = [Int16(variable(vtree.right))] 
    elseif typeof(vtree.left) == PlainVtreeLeafNode
        l = [Int16(variable(vtree.left))]
        r = bs2arr(vtree.right)
    else
        l = bs2arr(vtree.left)
        r = bs2arr(vtree.right)
    end    
    return [l,r]
end

lr (generic function with 1 method)

This is the standard clustering function of Juice. The only change are the assignments returned in output.

In [4]:
function clustering2(data, mix_num::Int64; maxiter=200)::Vector
    n = num_examples(data)
    if mix_num == 1
        return [data]
    end
    data = Matrix(data)
    R = kmeans(data', mix_num; maxiter=maxiter)
    @assert nclusters(R) == mix_num
    a = assignments(R)
    clustered_data = Vector()
    for k in 1 : mix_num
        push!(clustered_data, DataFrame(data[findall(x -> x == k, a), :]))
    end
    return [clustered_data,a]
end

clustering2 (generic function with 1 method)

Fixing col names with sub-datasets

In [5]:
function find_names(names_short,indexes_short,col_names)
    m=Any[]
    for i in indexes_short
        push!(m,findfirst(x->x==col_names[i], names_short))
    end
    return m
end

find_names (generic function with 1 method)

Core Learning Function

In [97]:
using LogicCircuits
using ProbabilisticCircuits
using DataFrames
using BenchmarkTools
using TikzPictures
using Statistics: mean
using LinearAlgebra: normalize
using Clustering: kmeans, nclusters, assignments
using UUIDs
TikzPictures.standaloneWorkaround(true)
import Printf
using LogicCircuits.LoadSave: SDDElement, 
    PSDDElement, 
    save_lines,
    get_vtree2id,
    get_node2id,
    parse_psdd_file, 
    PsddHeaderLine, 
    LcHeaderLine, 
    get_nodes_level

NCLUSTERS = 3
MINCLUSTERSIZE = 3
LOGS = true
VERBOSE = true


function bs2arr(bs)
    y = variables(bs) # parsing the indexes of the variables on the right DEBUG: case of a single var
    yy = (x->collect.(x)).(y)
    b = zeros(Int8,size(yy,1))
    for i = 1:size(yy,1)
        b[i] = yy[i][1]
    end
    return b
end

function lr(vtree)
    if typeof(vtree.left) == PlainVtreeLeafNode && typeof(vtree.right) == PlainVtreeLeafNode
        l = [Int16(variable(vtree.left))]
        r = [Int16(variable(vtree.right))] 
    elseif typeof(vtree.left) == PlainVtreeLeafNode
        l = [Int16(variable(vtree.left))]
        r = bs2arr(vtree.right)
    else
        l = bs2arr(vtree.left)
        r = bs2arr(vtree.right)
    end    
    return [l,r]
end

function clustering2(data, mix_num::Int64; maxiter=200)::Vector
    n = num_examples(data)
    if mix_num == 1
        return [data]
    end
    data = Matrix(data)
    R = kmeans(data', mix_num; maxiter=maxiter)
    @assert nclusters(R) == mix_num
    a = assignments(R)
    clustered_data = Vector()
    for k in 1 : mix_num
        push!(clustered_data, DataFrame(data[findall(x -> x == k, a), :]))
    end
    return [clustered_data,a]
end

function find_names(names_short,indexes_short,col_names)
    m=Any[]
    for i in indexes_short
        push!(m,findfirst(x->x==col_names[i], names_short))
    end
    return m
end


function LearnDecision(d,v,myrows,titles,parent=1, element=-1, sibling=-1)
    node_id = rand(Int,1)[1]
    psdd_node_id = length(myrows)+1 # id to identify the psdd nodes
    vtree_node_id = Int16(DICT_VTREE[v])+1 # id for the vtree nodes
    # size(d,1) > 0 ? nothing : throw(AssertionError("Empty dataset"))
    if typeof(v) == PlainVtreeLeafNode # for leav nodes a L or a T should be prepared
        var = Int16(variable(v)) # variable as in the vtree DEBUG check if -1
        nt = count(i->(i == true), d[:,1]) # number of true states in the first (unique) col of db d
        if nt == size(d,1) # all the elements true, the literal is +L
            myrow = ["L",psdd_node_id,vtree_node_id,var,parent,element,sibling]
        elseif nt == 0 # all false, the literal is -L
            myrow = ["L",psdd_node_id,vtree_node_id,-var,parent,element,sibling]            
        else # some true and some false, Top with a weight
            p = LOGS ? log(nt/size(d,1)) : nt/size(d,1) # P(True) or its Log
            myrow = ["T",psdd_node_id,vtree_node_id,var,p,parent,element,sibling]
        end
        append!(myrows,[myrow])
    else
        variables = [find_names(names(d),vrb,titles)  for vrb in lr(v)] # fixing names 
        if size(variables[1],1) == 1 # is single variable on the lft
            dd = [filter(row -> row[variables[1][1]] == i, d) for i in 0:1]
            d_left = [dd[i][:,variables[1]] for i in 1:2]
            d_right = [dd[i][:,variables[2]] for i in 1:2]
            p = [size(dd[i],1)/size(d,1) for i in 1:2]
            myrow = ["D",node_id,vtree_node_id]
            i = 1
            for c = 1:2
                if p[c] > 0.0
                    LearnDecision(d_left[c],v.left,myrows,titles,node_id,"prime",i)
                    LearnDecision(d_right[c],v.right,myrows,titles,node_id,"sub",i)
                    pc = LOGS ? log(p[c]) : p[c]
                    append!(myrow,["","",pc])
                    i += 1
                end
            end
            append!(myrow,[parent,element,sibling])
            insert!(myrow, 4, i-1)
            append!(myrows,[myrow])
        else
            if size(d,1) > MINCLUSTERSIZE
                d_left = d[:,variables[1]]
                d_right = d[:,variables[2]]
                clsts2 = clustering2(d_left,NCLUSTERS)
                clsts = clsts2[1]
                assignments = clsts2[2]
                p = [size(clsts[c],1)/size(d,1) for c = 1:NCLUSTERS]
                #psdd_node_id = length(myrows) # id to identify the psdd nodes
                myrow = ["D",node_id,vtree_node_id]
                i = 1
                for c = 1:NCLUSTERS
                    if p[c] > 0
                        # LOGS
                        rename!(clsts[c],names(d_left))    
                        LearnDecision(clsts[c],v.left,myrows,titles,node_id,"prime",i)
                        d2 = d_right[findall(x->x==c, assignments),:]
                        LearnDecision(d2,v.right,myrows,titles,node_id,"sub",i)
                        pc = LOGS ? log(p[c]) : p[c]
                        append!(myrow,["","",pc])
                        i += 1
                    end
                end
                append!(myrow,[parent,element,sibling])
                insert!(myrow, 4, i-1)
                append!(myrows,[myrow])
            else
                d_left = d[:,variables[1]]
                d_right = d[:,variables[2]]
                psdd_node_id = -1000
                myrow = ["D",node_id,vtree_node_id,1,"","",0.0,parent,element,1]
                LearnDecision(d_left,v.left,myrows,titles,node_id,"prime",1)
                LearnDecision(d_right,v.right,myrows,titles,node_id,"sub",1)
                #myrow[2] = length(myrows)
                append!(myrows,[myrow])
            end
        end
    end
    return myrows
end


function cleaner2(rows)
    dict = Dict()
    dict[-1] = -1
    dict[1] = 1
    dict[0] = 0
    for (r,row) in enumerate(rows)
        if row[1] == "D"
            dict[row[2]] = r
        end
    end
    for (r,row) in enumerate(rows)
        row[length(row)-2] = dict[row[length(row)-2]]
        if row[1] == "D"
            row[2] = dict[row[2]]
        end
        rows[r] = row #println(row[length(row)-2]," <> ",dict[row[length(row)-2]])
    end    
    for (r,row) in enumerate(rows)
        if row[1] == "D"
            goodrow = row
            candidates = Any[]
            pp = row[2]
            for row2 in rows
                if pp == row2[length(row2)-2]
                    append!(candidates,[row2])
                end
            end
            for candidate in candidates
                candidate_id = candidate[2]
                kk = length(candidate)
                sib = candidate[kk]
                if candidate[kk-1] == "prime"
                    goodrow[5+(sib-1)*3] = candidate_id
                else
                    goodrow[5+(sib-1)*3+1] = candidate_id
                end
            end
            goodrow[2] = r
            rows[r] = goodrow #[1:(length(row)]       
        end
    end
    return rows
end




function cleaner(rows)
    for (r,row) in enumerate(rows)
        if row[1] == "D"
            goodrow = row
            candidates = Any[]
            for row2 in rows
                if pp == row2[length(row2)-2]
                    append!(candidates,[row2])
                end
            end
            for candidate in candidates
                println(candidate)
                candidate_id = candidate[2]
                kk = length(candidate)
                sib = candidate[kk]
                if candidate[kk-1] == "prime"
                    goodrow[5+(sib-1)*3] = candidate_id
                end
                if candidate[kk-1] == "sub"
                #else
                    goodrow[5+(sib-1)*3+1] = candidate_id
                end
            end
            rows[r] = goodrow
            #if row[6] ==  ""
            #    println("===i==")
            #    for aa in candidates
            #        println(aa)
            #    end
            #    println("-----")
            #end
            #if length(row) < 3
            #    println("====")
            #    for cc in candidates
            #        println(candidates)
            #    end
        end
        #println(row)
        #println("====e====")

    end
    #end
    return rows
end


train_x, valid_x, test_x = twenty_datasets("nltcs") #plants
pc_cl, vtree_cl = learn_chow_liu_tree_circuit(train_x)

nd = 1000
d = train_x[1:nd,:] # Using fewer data for quicker tests
vtree = vtree_cl

DICT_VTREE = get_vtree2id(vtree);
pc = LearnDecision(d,vtree,[],names(d));
#io = open("/Users/install/row.psdd", "w")
#for o in pc
#    println((o[1:length(o)]," "))
#    #println(io,join(o[1:length(o)]," "))
#end
#a = "Ciao"
pc2 = cleaner2(pc);
io = open("/Users/install/psdd.psdd", "w")
for o in reverse(pc2)
    aa = join(o[1:length(o)]," ")
    #println(join(o[1:length(o)]," <> "))
    println(io,join(o[1:length(o)-3]," "))
    if aa[1:1] == "D"
        tokens = split(aa)
        head_ints::Vector{UInt32} = map(x->parse(UInt32,x),tokens[2:4])
        println(head_ints)
        println(aa)
        for (p,s,w) in Iterators.partition(tokens[5:end],3)
            println(p," ",s," ",w)
        end
    end
end
close(io)
println("finito")
#pc3 = load_prob_circuit("/Users/install/file2.psdd")
#plot(pc3)


UInt32[0x00000639, 0x0000001f, 0x00000002]
D 1593 31 2 1 968 -0.20702416943432653 969 1592 -1.6766466621275504 1 -1 -1
1 968 -0.20702416943432653
969 1592 -1.6766466621275504
1 -1 -1
UInt32[0x00000638, 0x0000001e, 0x00000003]
D 1592 30 3 1074 1203 -0.7424722471224467 1304 1389 -2.0956144009254367 1482 1591 -0.9136205033182762 1593 sub 2
1074 1203 -0.7424722471224467
1304 1389 -2.0956144009254367
1482 1591 -0.9136205033182762
1593 sub 2
UInt32[0x00000637, 0x0000001d, 0x00000002]
D 1591 29 2 1483 1544 -0.7911275889201491 1545 1590 -0.6039160468320027 1592 sub 3
1483 1544 -0.7911275889201491
1545 1590 -0.6039160468320027
1592 sub 3
UInt32[0x00000636, 0x0000001c, 0x00000003]
D 1590 28 3 1560 1561 -0.2478361639045812 1570 1571 -3.0204248861443626 1588 1589 -1.7676619176489945 1591 sub 2
1560 1561 -0.2478361639045812
1570 1571 -3.0204248861443626
1588 1589 -1.7676619176489945
1591 sub 2
UInt32[0x00000634, 0x0000001a, 0x00000002]
D 1588 26 2 1572 1579 -1.252762968495368 1580 1587 -0.336472236

UInt32[0x00000579, 0x0000000d, 0x00000001]
D 1401 13 1 1395 1400 0.0 1412 sub 1
1395 1400 0.0
1412 sub 1
UInt32[0x00000578, 0x0000000c, 0x00000001]
D 1400 12 1 1398 1399 0.0 1401 sub 1
1398 1399 0.0
1401 sub 1
UInt32[0x00000576, 0x0000000a, 0x00000001]
D 1398 10 1 1396 1397 0.0 1400 prime 1
1396 1397 0.0
1400 prime 1
UInt32[0x00000572, 0x00000006, 0x00000001]
D 1394 6 1 1392 1393 0.0 1412 prime 1
1392 1393 0.0
1412 prime 1
UInt32[0x0000056d, 0x0000001d, 0x00000002]
D 1389 29 2 1305 1356 -0.24512245803298496 1357 1388 -1.5260563034950494 1592 sub 2
1305 1356 -0.24512245803298496
1357 1388 -1.5260563034950494
1592 sub 2
UInt32[0x0000056c, 0x0000001c, 0x00000003]
D 1388 28 3 1366 1367 -1.6094379124341003 1376 1377 -1.6094379124341003 1386 1387 -0.5108256237659907 1389 sub 2
1366 1367 -1.6094379124341003
1376 1377 -1.6094379124341003
1386 1387 -0.5108256237659907
1389 sub 2
UInt32[0x0000056a, 0x0000001a, 0x00000001]
D 1386 26 1 1378 1385 0.0 1388 prime 3
1378 1385 0.0
1388 prime 3
UInt32[0

LoadError: BoundsError: attempt to access 2-element view(::Vector{SubString{String}}, 7:8) with eltype SubString{String} at index [3]

In [56]:
train_x, valid_x, test_x = twenty_datasets("nltcs") #plants
pc_cl, vtree_cl = learn_chow_liu_tree_circuit(train_x)

nd = 1000
d = train_x[1:nd,:] # Using fewer data for quicker tests
vtree = vtree_cl

DICT_VTREE = get_vtree2id(vtree)

pc = LearnDecision(d,vtree,[],names(d));
io = open("/Users/install/row.psdd", "w")
#for o in pc
#    println((o[1:length(o)]," "))
#    #println(io,join(o[1:length(o)]," "))
#end
#close(io)
pc2 = cleaner2(pc)
io = open("/Users/install/file2.psdd", "w")
for o in reverse(pc2)
    println(join(o[1:length(o)-3]," "))
    println(io,join(o[1:length(o)-3]," "))
end
close(io)
println("finito")
pc3 = load_prob_circuit("/Users/install/file2.psdd")
#plot(pc3)

D 1600 31 2 0 1005 -0.20702416943432653 1006 1599 -1.6766466621275504
D 1599 30 3 1073 1180 -0.9136205033182762 1319 1450 -0.6772317252540457 1523 1598 -2.3978952727983707
D 1598 29 2 1524 1565 -1.041453874828161 1566 1597 -0.4353180712578455
D 1597 28 3 1575 1576 -0.3184537311185346 1585 1586 -2.3978952727983707 1595 1596 -1.7047480922384253
T 1596 27 16 -0.6931471805599453
D 1595 26 1 1587 1594 0.0
D 1594 25 1 1590 1593 0.0
D 1593 24 1 1591 1592 0.0
L 1592 23 5
L 1591 22 14
D 1590 21 1 1588 1589 0.0
L 1589 20 12
L 1588 19 -11
L 1587 18 15
L 1586 27 -16
D 1585 26 1 1577 1584 0.0
D 1584 25 1 1580 1583 0.0
D 1583 24 1 1581 1582 0.0
L 1582 23 5
L 1581 22 14
D 1580 21 1 1578 1579 0.0
L 1579 20 12
L 1578 19 -11
L 1577 18 -15
T 1576 27 16 -0.4700036292457356
D 1575 26 1 1567 1574 0.0
D 1574 25 1 1570 1573 0.0
D 1573 24 1 1571 1572 0.0
L 1572 23 5
L 1571 22 14
D 1570 21 1 1568 1569 0.0
L 1569 20 12
L 1568 19 11
L 1567 18 15
L 1566 17 13
D 1565 28 3 1541 1542 -0.6931471805599453 1553 1554 -1.

L 1219 8 6
L 1218 7 -8
D 1217 6 1 1215 1216 0.0
L 1216 5 -1
L 1215 4 3
D 1214 13 2 1186 1199 -1.4469189829363254 1200 1213 -0.2682639865946794
D 1213 12 3 1203 1204 -0.7731898882334817 1207 1208 -0.9555114450274363 1211 1212 -1.8718021769015913
L 1212 11 10
D 1211 10 1 1209 1210 0.0
L 1210 9 -4
L 1209 8 -6
L 1208 11 10
D 1207 10 1 1205 1206 0.0
L 1206 9 -4
L 1205 8 6
L 1204 11 10
D 1203 10 1 1201 1202 0.0
L 1202 9 4
L 1201 8 6
L 1200 7 8
D 1199 12 3 1189 1190 -0.6931471805599453 1193 1194 -1.3862943611198906 1197 1198 -1.3862943611198906
L 1198 11 10
D 1197 10 1 1195 1196 0.0
L 1196 9 4
L 1195 8 -6
L 1194 11 10
D 1193 10 1 1191 1192 0.0
L 1192 9 4
L 1191 8 6
L 1190 11 10
D 1189 10 1 1187 1188 0.0
L 1188 9 -4
L 1187 8 6
L 1186 7 -8
D 1185 6 1 1183 1184 0.0
L 1184 5 -1
L 1183 4 -3
L 1182 3 -2
L 1181 2 -7
D 1180 29 2 1074 1117 -1.4842747694800944 1118 1179 -0.25704510298989114
D 1179 28 3 1143 1144 -0.1286173778220936 1167 1168 -2.268683541318364 1177 1178 -4.060443010546419
L 1178 27 -16

L 781 8 6
L 780 7 8
D 779 12 2 773 774 -0.6931471805599453 777 778 -0.6931471805599453
L 778 11 10
D 777 10 1 775 776 0.0
L 776 9 4
L 775 8 6
L 774 11 10
D 773 10 1 771 772 0.0
L 772 9 -4
L 771 8 6
L 770 7 -8
D 769 6 1 767 768 0.0
L 768 5 -1
L 767 4 -3
L 766 3 2
D 765 14 3 705 730 -0.7621400520468967 733 754 -0.7156200364120039 757 764 -3.1135153092103742
D 764 13 1 758 763 0.0
D 763 12 1 761 762 0.0
L 762 11 10
D 761 10 1 759 760 0.0
L 760 9 4
L 759 8 6
L 758 7 8
D 757 6 1 755 756 0.0
L 756 5 1
L 755 4 -3
D 754 13 2 734 739 -1.1451323043030026 740 753 -0.38299225225610584
D 753 12 3 743 744 -0.3101549283038396 747 748 -2.70805020110221 751 752 -1.6094379124341003
L 752 11 10
D 751 10 1 749 750 0.0
L 750 9 -4
L 749 8 6
L 748 11 10
D 747 10 1 745 746 0.0
L 746 9 4
L 745 8 -6
L 744 11 10
D 743 10 1 741 742 0.0
L 742 9 4
L 741 8 6
L 740 7 8
D 739 12 1 737 738 0.0
T 738 11 10 -0.15415067982725836
D 737 10 1 735 736 0.0
L 736 9 4
L 735 8 6
L 734 7 -8
D 733 6 1 731 732 0.0
L 732 5 -1
L 731 4

L 312 9 4
L 311 8 -6
L 310 11 10
D 309 10 1 307 308 0.0
L 308 9 -4
L 307 8 -6
L 306 7 -8
D 305 6 1 303 304 0.0
L 304 5 1
L 303 4 -3
D 302 13 2 282 301 -0.07410797215372185 296  -2.639057329615259
D 301 12 1 299 300 0.0
L 300 11 10
D 299 10 1 297 298 0.0
L 298 9 -4
L 297 8 -6
L 296 7 8
D 295 12 3 285 286 -1.466337068793427 289 290 -1.1786549963416462 293 294 -0.7731898882334817
L 294 11 10
D 293 10 1 291 292 0.0
L 292 9 -4
L 291 8 6
L 290 11 10
D 289 10 1 287 288 0.0
L 288 9 4
L 287 8 -6
L 286 11 10
D 285 10 1 283 284 0.0
L 284 9 -4
L 283 8 -6
L 282 7 -8
D 281 6 1 279 280 0.0
L 280 5 -1
L 279 4 3
D 278 13 2 250 263 -0.084557388028063 264 277 -2.512305623976115
D 277 12 3 267 268 -1.4350845252893227 271 272 -0.8472978603872037 275 276 -1.0986122886681098
L 276 11 10
D 275 10 1 273 274 0.0
L 274 9 -4
L 273 8 -6
L 272 11 10
D 271 10 1 269 270 0.0
L 270 9 -4
L 269 8 6
L 268 11 10
D 267 10 1 265 266 0.0
L 266 9 4
L 265 8 -6
L 264 7 8
D 263 12 3 253 254 -1.8613527610272502 257 258 -0.98363430

LoadError: BoundsError: attempt to access 2-element view(::Vector{SubString{String}}, 4:5) with eltype SubString{String} at index [3]

In [8]:
small_dataset = DataFrame(x1 = Bool[], x2 = Bool[], x3 = Bool[], x4 = Bool[])
nn = [10,32,22,44,11,4,21,10]
for i = 1:nn[1]
    push!(small_dataset, Dict(:x1 => false, :x2 => false, :x3 => false, :x4 => false))
end
for i = 1:nn[2]
    push!(small_dataset, Dict(:x1 => false, :x2 => false, :x3 => true, :x4 => true))
end
for i = 1:nn[3]
    push!(small_dataset, Dict(:x1 => false, :x2 => false, :x3 => false, :x4 => false))
end
for i = 1:nn[4]
    push!(small_dataset, Dict(:x1 => true, :x2 => false, :x3 => false, :x4 => true))
end
for i = 1:nn[5]
    push!(small_dataset, Dict(:x1 => false, :x2 => true, :x3 => true, :x4 => true))
end
for i = 1:nn[6]
    push!(small_dataset, Dict(:x1 => false, :x2 => true, :x3 => true, :x4 => false))
end
for i = 1:nn[7]
    push!(small_dataset, Dict(:x1 => true, :x2 => true, :x3 => false, :x4 => true))
end
for i = 1:nn[8]
    push!(small_dataset, Dict(:x1 => true, :x2 => true, :x3 => true, :x4 => false))
end

small_vtree = Vtree(4, :balanced)
plot(small_vtree)
DICT_VTREE = get_vtree2id(small_vtree)

pc_small = LearnDecision(small_dataset,small_vtree,[],names(small_dataset))
pc_small_clean = cleaner(pc_small)
io = open("/Users/install/small.psdd", "w")
for o in reverse(pc_small_clean)
    #println(join(o[1:length(o)-3]," "))
    println(io,join(o[1:length(o)-3]," "))
end
close(io)
pc_small_true = load_prob_circuit("/Users/install/small.psdd")
plot(pc)


LoadError: MethodError: no method matching plot(::Vector{Any})
[0mClosest candidates are:
[0m  plot(::Any, [91m::TikzGraphs.Layouts.Layout[39m) at /Users/install/.julia/packages/TikzGraphs/d963V/src/TikzGraphs.jl:44
[0m  plot(::Any, [91m::TikzGraphs.Layouts.Layout[39m, [91m::Vector{T}[39m; args...) where T<:AbstractString at /Users/install/.julia/packages/TikzGraphs/d963V/src/TikzGraphs.jl:44
[0m  plot(::Any, [91m::Vector{T}[39m; args...) where T<:AbstractString at /Users/install/.julia/packages/TikzGraphs/d963V/src/TikzGraphs.jl:45
[0m  ...

In [9]:
#file = zoo_psdd_file("little_4var.psdd")
#print(file)
#prob_circuit = load_prob_circuit(file);
#rows = ["c 1","c 2"]
#println(variables(vtree))
#println(isinner(vtree.right))
#println(varsubset(vtree.right,vtree))
#println(root(vtree.left))
#println(get_vtree2id(vtree.left))
#dd = get_vtree2id(vtree)
#print(dd[vtree.right])
#print(vr.parent)
#I'm long: $x, but I'm alright: $(@sprintf("%.2f", x))")
#df.x1 = 1:16
#println(df)
#println(train_x)
#plot(vtree_cl)
#println(df2)
#DICT_VTREE = get_vtree2id(v)
#println(typeof(v))
#println(typeof(vtree))
#println(v)
#println(vtree)
#DB_TITLE = names(df)
#println(DICT_VTREE)
#println(DB_TITLE)